# CASCADING AI CODING CHALLLENGE

## Installation of packages and imports

In [231]:
!pip install camelot-py
!apt install ghostscript python3-tk
!pip install ghostscript
!pip install transformers torch pandas scikit-learn numpy requests
!pip install fuzzywuzzy plotly

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.6).
python3-tk is already the newest version (3.10.8-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [232]:
import pandas as pd
import camelot
import numpy as np
import requests
import json
import plotly.graph_objects as go
from fuzzywuzzy import process
import plotly.express as px

## Data Preprocessing

In [233]:
tables1 = camelot.read_pdf('/content/pdf1.pdf', flag_size = True, pages='all')
if len(tables1) == 0:
  tables1 = camelot.read_pdf('/content/pdf1.pdf',  flavor='stream', flag_size = False, pages='all')

tables2 = camelot.read_pdf('/content/pdf2.pdf', flag_size = True, pages='all')
if len(tables2) == 0:
  tables2 = camelot.read_pdf('/content/pdf2.pdf',  flavor='stream', flag_size = False, pages='all')
tables3 = camelot.read_pdf('/content/pdf3.pdf',  flavor='stream', flag_size = False, pages='all')
tables4 = camelot.read_pdf('/content/pdf4.pdf',  flavor='stream', flag_size = False, pages='all')

/usr/local/lib/python3.10/dist-packages/camelot/parsers/stream.py:365: UserWarning:

No tables found in table area 1



### IDFC FIRST BANK

In [236]:
df1 = pd.DataFrame()
for i in range(len(tables1)):
  temp_df = tables1[i].df
  new_header = temp_df.iloc[0] # Grab the first row for the header
  temp_df = temp_df[1:] # Take the data less the header row
  temp_df.columns = new_header # Set the header row as the df header
  # Reset the index of the DataFrame
  temp_df.reset_index(drop=True, inplace=True)
  df1= pd.concat([df1, temp_df], ignore_index=True)
df1 = df1.drop('Value Date', axis=1)
df1 = df1.drop('Cheque No.', axis=1)

In [237]:
df1['Transaction  Date'] = pd.to_datetime(df1['Transaction  Date'], errors='coerce')
df1['Debit'] = df1['Debit'].str.replace(',', '', regex=False)
df1['Credit'] = df1['Credit'].str.replace(',', '', regex=False)
df1['Balance'] = df1['Balance'].str.replace(',', '', regex=False)
df1['Debit'] = pd.to_numeric(df1['Debit'])
df1['Credit'] = pd.to_numeric(df1['Credit'])
df1['Balance'] = pd.to_numeric(df1['Balance'])
df1


,Transaction Date,Particulars,Debit,Credit,Balance
0,2018-04-01,IMPS-MOB/Fund Trf/809119519742/Oma \nRam/BENE ...,NaN,1.0,22.62
1,2018-04-01,IMPS-MOB/Fund Trf/809119553510/Oma \nRam/2000,NaN,2000.0,2022.62
2,2018-04-03,NACH/TP ACH Bajaj Finanac/88551679,1912.0,NaN,110.62
3,2018-04-04,APB-INW/HP644950 000009 3003,NaN,170.2,280.82
4,2018-04-04,POS-VISA/KARUN MEDICAL/809414290216,75.0,NaN,205.82
...,...,...,...,...,...
199,2019-03-13,POS-VISA/COST TO COST/907212362533,460.0,NaN,2196.12
200,2019-03-13,UPI/MOB/907223556339/Verified Merch \nant,160.0,NaN,2036.12
201,2019-03-14,POS-VISA/POORVIKA/907309887397,1000.0,NaN,1036.12
202,2019-03-15,ATM-NFS/CASH WITHDRAWAL/+OPERA HOUS \nE/907414...,1000.0,NaN,36.12


### LLYODS BANK

In [238]:
df2 = tables2[1].df[2:]
new_header = df2.iloc[0] # Grab the first row for the header
df2 = df2[1:] # Take the data less the header row
df2.columns = new_header # Set the header row as the df header
# Reset the index of the DataFrame
df2.reset_index(drop=True, inplace=True)

In [239]:
tempdf2 = df2.replace(r'\s+', "", regex=True)
copydf = tempdf2.copy()

In [240]:
def merge_rows(df):
    i = 0
    while i < len(df) - 1:
        flag = False
        if df.loc[i, "Type"] == "":
            df.loc[i, 'Type'] = df.loc[i + 1, 'Type']
            flag = True
        if df.loc[i, "Description"] == "":
            df.loc[i, 'Description'] = df.loc[i + 1, 'Description']
            flag = True
        if df.loc[i, "Date"] == "":
            df.loc[i, 'Date'] = df.loc[i + 1, 'Date']
            flag = True
        if df.loc[i, "In (£)"] == "" and df.loc[i, "Out (£)"] == "":
            df.loc[i, 'In (£)'] = df.loc[i + 1, 'In (£)']
            df.loc[i, 'Out (£)'] = df.loc[i + 1, 'Out (£)']
            flag = True
        if df.loc[i, "Balance (£)"] == "" or df.loc[i, "Balance (£)"] is None:
            df.loc[i, 'Balance (£)'] = df.loc[i + 1, 'Balance (£)']
            flag = True

        if flag:
            df = df.drop(i + 1).reset_index(drop=True)
        else:
            i += 1

    return df

merged_df = merge_rows(tempdf2)
df2 = merged_df

In [241]:
df2['Date'] = pd.to_datetime(df2['Date'], errors='coerce')
df2['Out (£)'] = df2['Out (£)'].str.replace(',', '', regex=False)
df2['In (£)'] = df2['In (£)'].str.replace(',', '', regex=False)
df2['Balance (£)'] = df2['Balance (£)'].str.replace(',', '', regex=False)
df2['Out (£)'] = pd.to_numeric(df2['Out (£)'])
df2['In (£)'] = pd.to_numeric(df2['In (£)'])
df2['Balance (£)'] = pd.to_numeric(df2['Balance (£)'])
df2

2,Date,Description,Type,In (£),Out (£),Balance (£)
0,2019-09-04,LNKCOOPERATIVESWCD482108DEC19,CPT,NaN,300.00,873.56
1,2019-09-09,PADAMCZUKKASA7420,TFR,50.0,NaN,923.56
2,2019-09-11,BARCLAYCARDCD7420,DEB,NaN,2.50,921.06
3,2019-09-15,WESTERNVILLACD7420,DEB,NaN,240.00,681.06
4,2019-09-16,WINELEAFCD7420,DEB,NaN,26.69,654.37
5,2019-09-17,LNKCOOPERATIVESWCD742006DEC19,CPT,NaN,200.00,454.37
6,2019-09-18,LVLIFE03592291015W,DD,NaN,33.03,421.34
7,2019-09-22,PARKFOODANDWINECD7420,DEB,NaN,30.46,390.88
8,2019-09-22,LNKCOOPERATIVESWCD742005DEC19,CPT,NaN,20.00,370.88
9,2019-09-25,LNKCOOPERATIVESWCD742003DEC19,CPT,NaN,200.00,170.88


### US BANK

In [242]:
credits = tables3[1].df.iloc[3:32]
new_header = credits.iloc[0] # Grab the first row for the header
credits = credits[1:] # Take the data less the header row
credits.columns = new_header # Set the header row as the df header
# Reset the index of the DataFrame
credits.reset_index(drop=True, inplace=True)
debits = tables3[4].df.iloc[3:15]
new_header = ["1", "2", "3", "4", "5", "6"]
debits.columns = new_header
debits.reset_index(drop=True, inplace=True)
other_debits = tables3[5].df.iloc[13:29]
other_credits = tables3[5].df.iloc[4:10]
new_header = ["1", "2", "3"]
other_credits.columns = new_header
other_credits.reset_index(drop=True, inplace=True)
cheques = tables3[5].df.iloc[49:55]
cheques2 = tables3[7].df.iloc[5:16]
cheques2.drop([5], inplace=True)
new_header = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
cheques2.columns = new_header
cheques2.reset_index(drop=True, inplace=True)

<ipython-input-242-b14d8fadc774>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [243]:
c1, d1, od, oc, ch1, ch2p1, ch2p2 = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
c1["Date"] = credits["Date"]
c1["Type"] = credits[""]
c1["Credit"] = credits["Amount"]
d1["Date"] = debits["1"]
d1["Type"] = debits["3"]
d1["Debit"] = debits["6"]
od["Date"] = ["Oct 2 2017", "Oct 2 2017", "Oct 4 2017", "Oct 6 2017", "Oct 10 2017"]
od["Type"] = ["PANERA BREAD", "CHICK-FIL-A", "MO SEC OF STATE", "FREDPRYOR CAREER", "THE PLUMBING PRO"]
od["Debit"] = ["7.00", "7.02", "51.25", "149.00", "372.00"]
oc["Date"] = other_credits["1"]
oc["Type"] = ["CHEQUE", "", "CHEQUE", "",  "CHEQUE", ""]
oc["Credit"] = other_credits["3"]
ch1["Date"] = ["Oct 16 2017", "Oct 2 2017", "Oct 10 2017", "Oct 2 2017", "Oct 11 2017", "Oct 11 2017", "Oct 17 2017", "Oct 6 2017", "Oct 11 2017", "Oct 16 2017"]
ch1["Type"] = ["CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE"]
ch1["Debit"] = ["3777.34", "1146.08", "1146.08", "70.00", "200.00", "25000.00", "320.0", "228.00", "754.99", "33.23"]
ch2p1["Date"] = cheques2["2"]
ch2p1["Debit"] = cheques2["5"]
ch2p1["Type"] = ["CHEQUE","CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE"]
ch2p2["Date"] = cheques2["7"]
ch2p2["Debit"] = cheques2["10"]
ch2p2["Type"] = ["CHEQUE","CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE", "CHEQUE"]

In [244]:
c1 = c1.drop(c1.index[1::2]).reset_index(drop=True)
c1['Date'] = c1['Date'].str.replace("Electronic Deposit", "", regex=False)
c1['Date'] = c1['Date'] + ' 2017'
d1 = d1[d1['Date'] != '']
d1['Date'] = d1['Date'].str.replace(" Branch Account Transfer", "", regex=False)
d1['Debit'] = d1['Debit'].str.replace("-", "", regex=False)
d1.reset_index(inplace=True)
d1["Type"][1] = "WEB PYMNT"
d1["Type"][5] = "INS IN"
d1['Date'] = d1['Date'] + ' 2017'
d1.drop('index', axis=1, inplace=True)
oc = oc.drop(oc.index[1::2]).reset_index(drop=True)
oc['Date'] = oc['Date'].str.replace("Electronic Deposit", "", regex=False)
oc['Date'] = oc['Date'] + ' 2017'
ch2p1['Date'] = ch2p1['Date'].str.replace("\n", " ", regex=False)
ch2p1['Date'] = ch2p1['Date'] + ' 2017'
ch2p2 = ch2p2.drop(ch2p2.index[-1])
ch2p2['Date'] = ch2p2['Date'] + ' 2017'

<ipython-input-244-f4cf14731e5b>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-244-f4cf14731e5b>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-244-f4cf14731e5b>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-244-f4cf14731e5b>:9: SettingWithCopyWarning:


A value is trying

In [245]:
merged_df = pd.concat([d1, c1, oc, od, ch1, ch2p1, ch2p2], ignore_index=True, sort=False)
new_row = {'Date': 'Oct 2 2017', 'Type': "Opening Balance", 'Credit': '10,487.68', }

# Convert the new row into a DataFrame
new_row_df = pd.DataFrame([new_row])

# Concatenate the new row DataFrame with the original DataFrame
merged_df = pd.concat([new_row_df, merged_df], ignore_index=True)

In [246]:
merged_df= merged_df.fillna("")
merged_df['Date'] = pd.to_datetime(merged_df['Date'], errors='coerce')
merged_df['Debit'] = merged_df['Debit'].str.replace(',', '', regex=False)
merged_df['Credit'] = merged_df['Credit'].str.replace(',', '', regex=False)
merged_df['Debit'] = pd.to_numeric(merged_df['Debit'])
merged_df['Credit'] = pd.to_numeric(merged_df['Credit'])
merged_df = merged_df.sort_values(by='Date')
df3 = merged_df
df3

,Date,Type,Credit,Debit
0,2017-10-02,Opening Balance,10487.68,NaN
34,2017-10-02,CHEQUE,NaN,70.00
32,2017-10-02,CHEQUE,NaN,1146.08
27,2017-10-02,CHICK-FIL-A,NaN,7.02
26,2017-10-02,PANERA BREAD,NaN,7.00
9,2017-10-02,From 36 TREAS 310,7265.00,NaN
10,2017-10-03,From 36 TREAS 310,6400.00,NaN
43,2017-10-04,CHEQUE,NaN,3000.00
28,2017-10-04,MO SEC OF STATE,NaN,51.25
11,2017-10-04,From CGS ADMINISTATOR,11911.98,NaN


### Commonwealth Bank

In [249]:
t1 = tables4[1].df[2:]
t2 = tables4[2].df
new_header = t1.iloc[0] # Grab the first row for the header
t1 = t1[1:] # Take the data less the header row
t1.columns = new_header # Set the header row as the df header
t1.reset_index(drop=True, inplace=True)
new_header = t2.iloc[0] # Grab the first row for the header
t2 = t2[1:] # Take the data less the header row
t2.columns = new_header # Set the header row as the df header
t2.reset_index(drop=True, inplace=True)
df4= pd.concat([t1, t2], ignore_index=True)

In [250]:
tempdf4 = df4.replace(r'\s+', "", regex=True)
d4copy = tempdf4.copy()
column_names = d4copy.columns
column_names_list = column_names.tolist()
print(column_names_list)

['Date  Transaction', 'Debit', 'Credit', 'Balance']


In [251]:
def merge_rows(df):
    i = 0
    while i < len(df) - 1:
        flag = False
        if df.loc[i, "Debit"] == "" and df.loc[i, "Credit"] == "":
            df.loc[i, 'Debit'] = df.loc[i + 1, 'Debit']
            df.loc[i, 'Credit'] = df.loc[i + 1, 'Credit']
            flag = True
        if df.loc[i, "Balance"] == "":
            df.loc[i, 'Balance'] = df.loc[i + 1, 'Balance']
            flag = True
        if flag:
          df.loc[i, "Date  Transaction"] +=  df.loc[i+1, "Date  Transaction"]
          df = df.drop(i + 1).reset_index(drop=True)
        else:
            i += 1
    return df

merged_df = merge_rows(d4copy)

In [252]:
# Extract the Date and Transaction into separate columns
df4copy = merged_df.copy()
df4copy[['Date', 'Transaction']] = df4copy[df4copy.columns[0]].str.extract(r'(\d{2}[A-Za-z]{3})(.*)')

# Drop the original 'Date Transaction' column
df4copy = df4copy.drop('Date  Transaction', axis=1)
df4copy['Balance'] = df4copy['Balance'].str.replace('$', '', regex=False)
df4copy['Balance'] = df4copy['Balance'].str.replace('CR', '', regex=False)
df4 = df4copy

In [253]:
def assign_year(date_str):
    if date_str[-3:] in ['Oct', 'Nov', 'Dec']:
        return date_str + '2017'
    elif date_str[-3:] == 'Jan':
        return date_str + '2018'
    else:
        return date_str  # Or handle other cases as needed

# Apply the function to the 'Date' column
df4['Date'] = df4['Date'].apply(assign_year)

# Now convert to datetime
df4['Date'] = pd.to_datetime(df4['Date'], format='%d%b%Y', errors='coerce')

In [254]:
df4['Debit'] = df4['Debit'].str.replace(',', '', regex=False)
df4['Credit'] = df4['Credit'].str.replace(',', '', regex=False)
df4['Balance'] = df4['Balance'].str.replace(',', '', regex=False)
df4['Debit'] = pd.to_numeric(df4['Debit'])
df4['Credit'] = pd.to_numeric(df4['Credit'])
df4['Balance'] = pd.to_numeric(df4['Balance'])
df4

,Debit,Credit,Balance,Date,Transaction
0,50.00,NaN,2217.29,2017-10-14,2017OPENINGBALANCE15OctAlintaSalesPtyLtdNetBan...
1,1000.00,NaN,1167.29,2017-10-23,VIRGINMONEYNetBankBPAY495514724373501635385
2,70.00,NaN,1097.29,2017-10-23,Transfertoxx6832NetBank
3,56.42,NaN,1040.87,2017-10-23,DirectDebit180247SGIOMOT563142623171021
4,27.90,NaN,1012.97,2017-10-26,DirectDebit165969JETTSKINGSWAY201724473788
5,4.00,NaN,1008.97,2017-11-01,AccountFee
6,NaN,40.0,1048.97,2017-11-01,DirectCredit421520JESSICALAINGJL-Internet
7,NaN,1000.0,2048.97,2017-11-01,Transferfromxx8727CommBankapp
8,122.25,NaN,1926.72,2017-11-01,DirectDebit000702HBF-HEALTH1439123504
9,700.00,NaN,1226.72,2017-11-05,TransfertootherBankNetBankOronsay


## Insights into the Data


In [255]:
def gpt_analysis(df, prompt):
  # Prepare your data for sending (convert DataFrame to a string or a structured format)
  data_to_analyze = df.to_json(orient='records')

  # Set up the API URL and your API key
  api_url = 'https://api.openai.com/v1/chat/completions'
  headers = {
      'Content-Type': 'application/json',
      'Authorization': 'Bearer OPENAI_API_KEY'  # Replace with your actual API key
  }

  # Format your request data
  messages = [
      {"role": "system", "content": "You are a bank statements analyzer."},
      {"role": "user", "content": f"{prompt}: {data_to_analyze}"}
  ]

  payload = {
      "model": "gpt-4",
      "messages": messages
  }

  # Send the request
  response = requests.post(api_url, headers=headers, data=json.dumps(payload))

  # Check and process the response
  if response.status_code == 200:
      result = response.json()
      print("Analysis:", result['choices'][0]['message']['content'])
  else:
      print("Error:", response.status_code, response.text)

### IDFC FIRST BANK

In [256]:
# Assuming df2 is your dataframe after the preprocessing steps you mentioned
df1['MonthYear'] = df1['Transaction  Date'].dt.to_period('M').astype(str)

# Calculating the sum of credit and debit for each month
monthly_totals = df1.groupby('MonthYear').agg({'Credit': 'sum', 'Debit': 'sum'}).reset_index()

# Plotting with Plotly
fig = go.Figure()
fig.add_trace(go.Bar(x=monthly_totals['MonthYear'], y=monthly_totals['Credit'], name='Credit', marker_color='blue'))
fig.add_trace(go.Bar(x=monthly_totals['MonthYear'], y=monthly_totals['Debit'], name='Debit', marker_color='red'))

fig.update_layout(title='Monthly Total Credits and Debits',
                   xaxis_title='Month',
                   yaxis_title='Amount',
                   barmode='group')
fig.show()

In [257]:
# Calculate the average monthly balance
average_monthly_balance = df1.groupby('MonthYear')['Balance'].mean().reset_index()

# Plotting with Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=average_monthly_balance['MonthYear'], y=average_monthly_balance['Balance'],
                         mode='lines+markers', name='Average Monthly Balance'))

fig.update_layout(title='Average Monthly Balance per Month',
                   xaxis_title='Month',
                   yaxis_title='Average Balance',
                   xaxis=dict(showline=True, showgrid=False, showticklabels=True),
                   yaxis=dict(showline=True, showgrid=False, showticklabels=True),
                   plot_bgcolor='white')

fig.show()

In [296]:
top_10_debit = df1.sort_values(by='Debit', ascending=False).head(10)
top_10_credit = df1.sort_values(by='Credit', ascending=False).head(10)

In [259]:
prompt = "Give me repeating keywords from the Particular column on what type of transactions these are"

gpt_analysis(top_10_debit, prompt)

Analysis: The repeating keywords from the Particular column that describe the type of transactions are:

1. "CHQ Paid" - Cheque Payment
2. "BILLPAY" - Bill Payments 
3. "POS-VISA" - Point of Sale Visa Debit Card Transaction (likely purchases from a vendor)
4. "ATM-NFS\/CASH WITHDRAWAL" - Cash Withdrawal from ATM 
5. "IMPS-RIB\/Fund Trf" - Fund Transfer using Immediate Payment Service (IMPS) from your mobile (RIB indicates Retail Internet Banking)


In [293]:
keywords = ['Bill Pay', 'ATM', 'CHQ', 'Fund Trf', 'IMPS', 'POS', 'Visa']
threshold = 70

def contains_keywords(text):
    for word in text.split('/'):
        match = process.extractOne(word, keywords, score_cutoff=threshold)
        if match:
            return match[0]  # Return the matched keyword
    return None  # Return None if no match found

# Apply fuzzy matching
top_10_debit['Category'] = top_10_debit['Particulars'].apply(contains_keywords)

# Filter rows where a keyword was matched
debitst10 = top_10_debit[top_10_debit['Category'].notnull()]

In [294]:
# Creating a pie chart
fig = px.pie(debitst10, values='Debit', names='Category', title='Top Debit Transactions by Type')

# Update the layout for better readability
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=True)

# Display the pie chart
fig.show()

In [297]:
prompt = "Give me repeating keywords from the Particular column on what type of transactions these are"

gpt_analysis(top_10_credit, prompt)

Analysis: The repeating keywords from the 'Particular' column that can be seen are the types of transactions and they are:

- "NEFT"
- "IMPS-MOB/Fund Trf"
- "IMPS-INET/Fund Trf"
- "IMPS-BRC/Fund Trf" 

These represent different types of bank transactions. NEFT stands for National Electronic Funds Transfer, and IMPS represents Immediate Payment Service which could be done through mobile (MOB), internet (INET) or branch (BRC).


In [298]:
keywords = ['NEFT', 'MOB', 'Fund Trf', 'IMPS', 'INET', 'BRC']
threshold = 70

def contains_keywords(text):
    for word in text.split('/'):
        match = process.extractOne(word, keywords, score_cutoff=threshold)
        if match:
            return match[0]  # Return the matched keyword
    return None  # Return None if no match found
top_10_credit['Category'] = top_10_credit['Particulars'].apply(contains_keywords)
creditst10 = top_10_credit[top_10_credit['Category'].notnull()]

In [299]:
# Creating a pie chart
fig = px.pie(creditst10, values='Credit', names='Category', title='Top Credit Transactions by Type')

# Update the layout for better readability
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=True)

# Display the pie chart
fig.show()

In [279]:
df1 = df1.drop('MonthYear', axis=1)

prompt = "Tell me if this bank statement can be approved for a loan or not by giving YES or NO answers with for the given four points. If the answer is no NO for anyone of them, give an explanation. 1. If has good amount of credits 2. If debits are normal debit transactions 3. Overall good balances every month 4. If there are no overdraft fees "

gpt_analysis(df1[:150], prompt)

Analysis: 1. Good amount of credits - Yes
2. Normal debit transactions - Yes
3. Overall good balances every month - No. Explanation - The account balance seems to be quite inconsistent, with instances of the balance being very low. This indicates the person may be living paycheck to paycheck which may make repayment of the loan difficult.
4. No overdraft fees - Yes


### LLYODS BANK

In [272]:
import pandas as pd
import plotly.graph_objects as go

# Assuming df2 is your dataframe after the preprocessing steps you mentioned
df2['MonthYear'] = df2['Date'].dt.to_period('M').astype(str)

# Calculating the sum of credit and debit for each month
monthly_totals = df2.groupby('MonthYear').agg({'In (£)': 'sum', 'Out (£)': 'sum'}).reset_index()

# Plotting with Plotly
fig = go.Figure()
fig.add_trace(go.Bar(x=monthly_totals['MonthYear'], y=monthly_totals['In (£)'], name='In (£)', marker_color='blue'))
fig.add_trace(go.Bar(x=monthly_totals['MonthYear'], y=monthly_totals['Out (£)'], name='Out (£)', marker_color='red'))

fig.update_layout(title='Monthly Total Credits and Debits',
                   xaxis_title='Month',
                   yaxis_title='Amount',
                   barmode='group')
fig.show()

In [273]:
# Calculate the average monthly balance
average_monthly_balance = df2.groupby('MonthYear')['Balance (£)'].mean().reset_index()

# Plotting with Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=average_monthly_balance['MonthYear'], y=average_monthly_balance['Balance (£)'],
                         mode='lines+markers', name='Average Monthly Balance'))

fig.update_layout(title='Average Monthly Balance per Month',
                   xaxis_title='Month',
                   yaxis_title='Average Balance',
                   xaxis=dict(showline=True, showgrid=False, showticklabels=True),
                   yaxis=dict(showline=True, showgrid=False, showticklabels=True),
                   plot_bgcolor='white')

fig.show()

In [300]:
top_10_debit = df2.sort_values(by='Out (£)', ascending=False).head(10)
top_10_credit = df2.sort_values(by='In (£)', ascending=False).head(10)

In [301]:
debitst10 = top_10_debit[top_10_debit['Type'].notnull()]
creditst10 = top_10_credit[top_10_credit['Type'].notnull()]

In [276]:
fig = px.pie(debitst10, values='Out (£)', names='Type', title='Top Debit Transactions by Type')

# Update the layout for better readability
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=True)

# Display the pie chart
fig.show()

In [277]:
# Creating a pie chart
fig = px.pie(creditst10, values='In (£)', names='Type', title='Top Credit Transactions by Type')

# Update the layout for better readability
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=True)

# Display the pie chart
fig.show()

In [278]:
df2 = df2.drop('MonthYear', axis=1)

prompt = "Tell me if this bank statement can be approved for a loan or not by giving YES or NO answers with for the given four points. If the answer is no NO for anyone of them, give an explanation. 1. If has good amount of credits 2. If debits are normal debit transactions 3. Overall good balances every month 4. If there are no overdraft fees "

gpt_analysis(df2, prompt)

Analysis: 1. Good amount of credits: YES. The bank statement shows multiple credit transactions indicating a good amount of credits coming in on different dates. 

2. Normal debit transactions: YES. The bank statement appears to have normal debit transactions like grocery shopping (COOPERATIVESWCD7420), purchases (SPORTSDIRECT252CD7420), and card swipes at different shops (WINELEAFCD7420). 

3. Overall good balances every month: YES. The balance seems to be adequately maintained every month with the lowest balance being 153.45 and the highest balance reaching 2579.56. 

4. No overdraft fees: YES. There's no indication in the statement provided of any overdraft fees being levied. 

Therefore, according to this analysis, this bank statement can be approved for a loan.


### US BANK

In [302]:
import pandas as pd
import plotly.graph_objects as go

# Assuming df2 is your dataframe after the preprocessing steps you mentioned
df3['MonthYear'] = df3['Date'].dt.to_period('M').astype(str)

# Calculating the sum of credit and debit for each month
monthly_totals = df3.groupby('MonthYear').agg({'Credit': 'sum', 'Debit': 'sum'}).reset_index()

# Plotting with Plotly
fig = go.Figure()
fig.add_trace(go.Bar(x=monthly_totals['MonthYear'], y=monthly_totals['Credit'], name='Credit', marker_color='blue'))
fig.add_trace(go.Bar(x=monthly_totals['MonthYear'], y=monthly_totals['Debit'], name='Debit', marker_color='red'))

fig.update_layout(title='Monthly Total Credits and Debits',
                   xaxis_title='Month',
                   yaxis_title='Amount',
                   barmode='group')
fig.show()

In [303]:
top_10_debit = df3.sort_values(by='Debit', ascending=False).head(10)
top_10_credit = df3.sort_values(by='Credit', ascending=False).head(10)

In [283]:
prompt = "Give me repeating keywords from the Transaction column on what type of transactions these are"

gpt_analysis(top_10_debit, prompt)

Analysis: The repeating keywords in the Transaction column are "CHEQUE", "To Account", and "From". These indicate that the types of transactions being carried out are cheque debits, transfers to other accounts, and inflow from various sources respectively.


In [304]:
keywords = ['Cheque', 'To Account', 'PHILA INS CO']
threshold = 70

def contains_keywords(text):
    for word in text.split('/'):
        match = process.extractOne(word, keywords, score_cutoff=threshold)
        if match:
            return match[0]  # Return the matched keyword
    return None  # Return None if no match found

# Apply fuzzy matching
top_10_debit['Category'] = top_10_debit['Type'].apply(contains_keywords)

# Filter rows where a keyword was matched
debitst10 = top_10_debit[top_10_debit['Category'].notnull()]

In [305]:
# Creating a pie chart
fig = px.pie(debitst10, values='Debit', names='Category', title='Top Debit Transactions by Type')

# Update the layout for better readability
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=True)

# Display the pie chart
fig.show()

In [286]:
prompt = "Give me repeating keywords from the Transaction column on what type of transactions these are"

gpt_analysis(top_10_credit, prompt)

Analysis: The repeating keywords that describe the types of transactions are:

1. "From CGS ADMINISTATOR"
2. "CHEQUE"
3. "Opening Balance"
4. "From 36  TREAS 310"


In [308]:
keywords = ['CGS Administrater', 'Cheque', 'Treas']
threshold = 70

def contains_keywords(text):
    for word in text.split('/'):
        match = process.extractOne(word, keywords, score_cutoff=threshold)
        if match:
            return match[0]  # Return the matched keyword
    return None  # Return None if no match found

# Apply fuzzy matching
top_10_credit['Category'] = top_10_credit['Type'].apply(contains_keywords)

# Filter rows where a keyword was matched
creditst10 = top_10_credit[top_10_credit['Category'].notnull()]

In [309]:
# Creating a pie chart
fig = px.pie(creditst10, values='Credit', names='Category', title='Top Credit Transactions by Type')

# Update the layout for better readability
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=True)

# Display the pie chart
fig.show()

In [229]:
df3 = df3.drop('MonthYear', axis=1)

prompt = "Tell me if this bank statement can be approved for a loan or not by giving YES or NO answers with for the given four points. If the answer is no NO for anyone of them, give an explanation. 1. If has good amount of credits 2. If debits are normal debit transactions 3. Overall good balances every month 4. If there are no overdraft fees "

gpt_analysis(df3, prompt)

Analysis: 1. YES: The account has a good amount of credits coming in regularly from sources like "36 TREAS 310" and "CGS ADMINISTATOR"
2. YES: All the debits seem to be for normal transactions like cheque deductions, various purchases, and transfers
3. YES: The overall balance seems to be positive at the end of each month, although it would be beneficial to see closing balance data to confirm
4. YES: There aren't any overdraft fees recorded in this dataset. Therefore, the bank statement meets all the conditions for a loan approval.


### Commonwealth Bank

In [310]:
import pandas as pd
import plotly.graph_objects as go

# Assuming df2 is your dataframe after the preprocessing steps you mentioned
df4['MonthYear'] = df4['Date'].dt.to_period('M').astype(str)

# Calculating the sum of credit and debit for each month
monthly_totals = df4.groupby('MonthYear').agg({'Credit': 'sum', 'Debit': 'sum'}).reset_index()

# Plotting with Plotly
fig = go.Figure()
fig.add_trace(go.Bar(x=monthly_totals['MonthYear'], y=monthly_totals['Credit'], name='Credit', marker_color='blue'))
fig.add_trace(go.Bar(x=monthly_totals['MonthYear'], y=monthly_totals['Debit'], name='Debit', marker_color='red'))

fig.update_layout(title='Monthly Total Credits and Debits',
                   xaxis_title='Month',
                   yaxis_title='Amount',
                   barmode='group')
fig.show()

In [311]:
# Calculate the average monthly balance
average_monthly_balance = df4.groupby('MonthYear')['Balance'].mean().reset_index()

# Plotting with Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=average_monthly_balance['MonthYear'], y=average_monthly_balance['Balance'],
                         mode='lines+markers', name='Average Monthly Balance'))

fig.update_layout(title='Average Monthly Balance per Month',
                   xaxis_title='Month',
                   yaxis_title='Average Balance',
                   xaxis=dict(showline=True, showgrid=False, showticklabels=True),
                   yaxis=dict(showline=True, showgrid=False, showticklabels=True),
                   plot_bgcolor='white')

fig.show()

In [312]:
top_10_debit = df4.sort_values(by='Debit', ascending=False).head(10)
top_10_credit = df4.sort_values(by='Credit', ascending=False).head(10)

In [313]:
prompt = "Give me repeating keywords from the Transaction column on what type of transactions these are"

gpt_analysis(top_10_debit, prompt)

Analysis: The repeating keywords in the Transaction column indicating the type of transaction are: 

1. VIRGINMONEYNetBankBPAY
2. TAXOFFICEPAYMENTSNetBankBPAY
3. TransfertootherBankNetBank
4. DirectDebit
5. HBF-HEALTH
6. TELSTRACORPLTDNetBankBPAY 

These keywords represent the types of transactions such as direct debit, transfer to other bank and payments made using NetBank BPAY to Virgin Money, Tax Office, HBF-Health, and Telstra Corp Ltd.


In [318]:
keywords = ['Bankbpay', 'Health', 'Direct Debit', 'Transfer', 'Tax office']
threshold = 70

def contains_keywords(text):
    for word in text.split('/'):
        match = process.extractOne(word, keywords, score_cutoff=threshold)
        if match:
            return match[0]  # Return the matched keyword
    return None  # Return None if no match found

# Apply fuzzy matching
top_10_debit['Category'] = top_10_debit['Transaction'].apply(contains_keywords)

# Filter rows where a keyword was matched
debitst5 = top_10_debit[top_10_debit['Category'].notnull()]
debitst5

,Debit,Credit,Balance,Date,Transaction,MonthYear,Category
1,1000.00,NaN,1167.29,2017-10-23,VIRGINMONEYNetBankBPAY495514724373501635385,2017-10,Bankbpay
21,850.75,NaN,101.80,2017-12-06,TAXOFFICEPAYMENTSNetBankBPAY755565510040455084...,2017-12,Tax office
34,700.00,NaN,510.43,2018-01-05,TransfertootherBankNetBankOronsay,2018-01,Transfer
9,700.00,NaN,1226.72,2017-11-05,TransfertootherBankNetBankOronsay,2017-11,Transfer
20,700.00,NaN,952.55,2017-12-05,TransfertootherBankNetBankOronsay,2017-12,Transfer
13,190.70,NaN,836.70,2017-11-22,DirectDebit068093SYNERGYRETAILB257300733078,2017-11,Direct Debit
30,122.25,NaN,508.33,2018-01-02,DirectDebit000702HBF-HEALTH1439123504,2018-01,Health
8,122.25,NaN,1926.72,2017-11-01,DirectDebit000702HBF-HEALTH1439123504,2017-11,Health
19,122.25,NaN,1652.55,2017-12-01,DirectDebit000702HBF-HEALTH1439123504,2017-12,Health
35,115.00,NaN,395.43,2018-01-07,TELSTRACORPLTDNetBankBPAY237962000353972134,2018-01,Bankbpay


In [319]:
# Creating a pie chart
fig = px.pie(debitst5, values='Debit', names='Category', title='Top Debit Transactions by Type')

# Update the layout for better readability
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=True)

# Display the pie chart
fig.show()

In [320]:
prompt = "Give me repeating keywords from the Transaction column on what type of transactions these are"

gpt_analysis(top_10_credit, prompt)

Analysis: From analyzing the Transaction column, the repeating keywords indicating the types of transactions are :

1. "Transferfrom"
2. "NetBank"
3. "CommBankapp"
4. "DirectCredit"
5. "OPENINGBALANCE"
6. "NetBankBPAY"
7. "VIRGINMONEYNetBankBPAY"


In [322]:
keywords = ['Direct Credit', 'Bankbpay', 'Transfer']
threshold = 70

def contains_keywords(text):
    for word in text.split('/'):
        match = process.extractOne(word, keywords, score_cutoff=threshold)
        if match:
            return match[0]  # Return the matched keyword
    return None  # Return None if no match found
top_10_credit['Category'] = top_10_credit['Transaction'].apply(contains_keywords)
creditst10 = top_10_credit[top_10_credit['Category'].notnull()]

In [323]:
# Creating a pie chart
fig = px.pie(creditst10, values='Credit', names='Category', title='Top Credit Transactions by Type')

# Update the layout for better readability
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=True)

# Display the pie chart
fig.show()

In [324]:
df4 = df4.drop('MonthYear', axis=1)

prompt = "Tell me if this bank statement can be approved for a loan or not by giving YES or NO answers with for the given four points. If the answer is no NO for anyone of them, give an explanation. 1. If has good amount of credits 2. If debits are normal debit transactions 3. Overall good balances every month 4. If there are no overdraft fees "

gpt_analysis(df4, prompt)

Analysis: 1. YES, the account received good amounts of credits.
2. YES, there are normal debit transactions.
3. NO, based on the given transactions, we can see that there is a trend of account balance diminish every month. It seems like the account holder is depleting the account and not maintaining a good balance every month.
4. YES, there are no overdraft fees recorded in these transactions.


## Conclusion

I have conducted a comprehensive analysis of bank statements from the various banks given. This process began with meticulous data preprocessing, followed by the creation of several graphs to assess the financial status of each account. Additionally, I have leveraged the advanced capabilities of the GPT-4 model to analyze the data, providing insights crucial for making informed loan approval decisions based on these bank statements.

Looking ahead to potential production implementation, my plan is to develop a custom PDF processing solution for each bank. This will enable efficient extraction of text from PDFs into a structured and analyzable DataFrame format. Furthermore, I intend to utilize Streamlit for building an interactive interface. This interface will allow users to upload bank statements and, in return, receive detailed data visualizations and GPT-based recommendations.

Due to time constraints and other significant commitments, I have compiled this analysis into a notebook format that specifically focuses on bank-wise examination and assessment.